In [1]:
!pip install -q tf-keras

In [2]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00


In [3]:
import os, numpy as np, pandas as pd, torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, f1_score
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from sentence_transformers.cross_encoder import CrossEncoder
import os
os.environ["WANDB_DISABLED"] = "true"          
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  

2025-09-06 21:39:49.488238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757194789.692956      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757194789.758719      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
SEED = 67
VAL_SIZE = 0.20
EPOCHS = 2
TRAIN_BS = 64
EVAL_BS = 256
LR = 3e-5

TRAIN_PATH = "/kaggle/input/quora-train/train.csv"   
TEST_PATH  = "/kaggle/input/quora-train/test.csv"    
VALID_PATH = "valid.csv"                              
TRAIN_SPLIT_PATH = "new_train_split.csv"

rng = np.random.RandomState(SEED)
torch.manual_seed(SEED)

In [6]:
import numpy as np

train_full = pd.read_csv(TRAIN_PATH)
_ = pd.read_csv(TEST_PATH)

TEXT_COLS = ["question1", "question2"]
LABEL_COL = "is_duplicate"

train_full = train_full.dropna(subset=TEXT_COLS + [LABEL_COL]).copy()

for c in TEXT_COLS:
    train_full[c] = train_full[c].astype(str)

if os.path.exists(VALID_PATH) and os.path.exists(TRAIN_SPLIT_PATH):
    valid_df = pd.read_csv(VALID_PATH)
    train_df = pd.read_csv(TRAIN_SPLIT_PATH)
else:
    train_df, valid_df = train_test_split(
        train_full,
        test_size=VAL_SIZE,
        random_state=SEED,
        stratify=train_full[LABEL_COL]
    )
    train_df.to_csv(TRAIN_SPLIT_PATH, index=False)
    valid_df.to_csv(VALID_PATH, index=False)

# Safety: enforce string dtype post-split too
for df in (train_df, valid_df):
    for c in TEXT_COLS:
        df[c] = df[c].astype(str)
    df[LABEL_COL] = df[LABEL_COL].astype(int)


/tmp/ipykernel_19/576938904.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  _ = pd.read_csv(TEST_PATH)


In [7]:
def to_examples(df):
    return [
        InputExample(
            texts=[str(row["question1"]), str(row["question2"])],
            label=float(row["is_duplicate"])
        )
        for _, row in df.iterrows()
    ]

train_examples_full = to_examples(train_df)
valid_examples      = to_examples(valid_df)

In [8]:
def encode_pairwise_cosine(model, examples, batch_size=128, normalize=True):
    q1 = [ex.texts[0] for ex in examples]
    q2 = [ex.texts[1] for ex in examples]
    y  = np.array([int(round(ex.label)) for ex in examples], dtype=np.int32)

    emb1 = model.encode(q1, batch_size=batch_size, convert_to_numpy=True,
                        show_progress_bar=False, normalize_embeddings=normalize)
    emb2 = model.encode(q2, batch_size=batch_size, convert_to_numpy=True,
                        show_progress_bar=False, normalize_embeddings=normalize)
    sims = (emb1 * emb2).sum(axis=1) if normalize else \
           util.cos_sim(torch.tensor(emb1), torch.tensor(emb2)).diagonal().cpu().numpy()
    return sims.astype(np.float32), y

def tune_threshold(y_true, scores):
    precision, recall, thresholds = precision_recall_curve(y_true, scores)
    f1s = (2 * precision * recall) / np.clip(precision + recall, 1e-12, None)
    best_idx = np.nanargmax(f1s[1:]) + 1
    return float(thresholds[best_idx - 1]), float(f1s[best_idx])

In [9]:
# (1) Baseline:(no fine-tuning)
def eval_baseline_biencoder(model_name, eval_bs=EVAL_BS, max_len=96):
    model = SentenceTransformer(model_name)
    model.max_seq_length = max_len
    val_scores, yv = encode_pairwise_cosine(model, valid_examples, batch_size=eval_bs, normalize=True)
    thr, val_f1 = tune_threshold(yv, val_scores)
    print(f"[0-baseline:{model_name}] Val F1={val_f1:.4f} @ thr={thr:.3f}")
    return model_name, val_f1, thr

In [10]:
# Bi-encoder training
def run_biencoder(loss_type, base_model, epochs=EPOCHS, batch_size=TRAIN_BS, lr=LR, eval_bs=EVAL_BS, max_len=96):
    model = SentenceTransformer(base_model)
    model.max_seq_length = max_len

    if loss_type == "mnr":
        pos_train = [ex for ex in train_examples_full if int(round(ex.label)) == 1]
        train_loss   = losses.MultipleNegativesRankingLoss(model)
        train_loader = DataLoader(pos_train, shuffle=True, batch_size=batch_size)

    elif loss_type == "cos":
        cos_train    = [InputExample(texts=ex.texts, label=float(int(round(ex.label))))
                        for ex in train_examples_full]
        train_loss   = losses.CosineSimilarityLoss(model)
        train_loader = DataLoader(cos_train, shuffle=True, batch_size=batch_size)

    elif loss_type == "contrastive":
        ctr_train    = [InputExample(texts=ex.texts, label=float(int(round(ex.label))))
                        for ex in train_examples_full]
        train_loss   = losses.OnlineContrastiveLoss(
            model, distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE, margin=0.5
        )
        train_loader = DataLoader(ctr_train, shuffle=True, batch_size=batch_size)
    else:
        raise ValueError("loss_type must be one of: 'mnr', 'cos', 'contrastive'")

    warmup_steps = max(100, int(0.1 * len(train_loader) * epochs))
    model.fit(
        train_objectives=[(train_loader, train_loss)],
        epochs=epochs,
        warmup_steps=warmup_steps,
        optimizer_params={'lr': lr},
        show_progress_bar=True
    )

    val_scores, yv = encode_pairwise_cosine(model, valid_examples, batch_size=eval_bs, normalize=True)
    thr, val_f1 = tune_threshold(yv, val_scores)
    print(f"[bi-{loss_type}:{base_model}] Val F1={val_f1:.4f} @ thr={thr:.3f}")
    return model, val_f1, thr

In [11]:
from torch.utils.data import DataLoader
from sentence_transformers import InputExample
from sentence_transformers.cross_encoder import CrossEncoder
import numpy as np

def run_cross_encoder(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",
                      epochs=1, batch_size=64, warmup_ratio=0.1, max_len=96):
    # Use your downsample if defined; else full train_df
    df_tr = TRAIN_CE if "TRAIN_CE" in globals() else train_df

    # Build training samples
    train_samples = [
        InputExample(texts=[str(q1), str(q2)], label=float(lbl))
        for q1, q2, lbl in zip(df_tr["question1"], df_tr["question2"], df_tr["is_duplicate"])
    ]
    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)

    # Convert ratio → steps for this version of .fit()
    total_steps = max(1, len(train_dataloader) * epochs)
    warmup_steps = max(10, int(total_steps * warmup_ratio))

    # Train
    ce = CrossEncoder(model_name, num_labels=1, max_length=max_len)
    print(f"[cross] training on {len(train_samples):,} pairs | "
          f"steps/epoch={len(train_dataloader)} | warmup_steps={warmup_steps}", flush=True)

    ce.fit(
        train_dataloader=train_dataloader,
        epochs=epochs,
        warmup_steps=warmup_steps,
        show_progress_bar=True,
        # use_amp=True,  # uncomment if you want mixed precision
    )

    # Validate: predict → tune threshold → F1
    va_pairs  = list(zip(valid_df["question1"].astype(str).tolist(),
                         valid_df["question2"].astype(str).tolist()))
    va_labels = valid_df["is_duplicate"].astype(int).to_numpy()

    val_scores = ce.predict(va_pairs, batch_size=EVAL_BS)
    thr, val_f1 = tune_threshold(va_labels, val_scores)
    print(f"[cross-encoder:{model_name}] Val F1={val_f1:.4f} @ thr={thr:.3f}")
    return model_name, val_f1, thr


In [12]:
# Run all 5

# (1) Baseline (no FT)
base_name, base_val_f1, base_thr = eval_baseline_biencoder("sentence-transformers/all-MiniLM-L6-v2")

# (2) Bi-encoder: Cosine Similarity Loss
cos_model, cos_val_f1, cos_thr = run_biencoder(
    loss_type="cos",
    base_model="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    epochs=EPOCHS, batch_size=TRAIN_BS, lr=LR, max_len=96
)

# (3) Bi-encoder: Contrastive Loss
ctr_model, ctr_val_f1, ctr_thr = run_biencoder(
    loss_type="contrastive",
    base_model="sentence-transformers/paraphrase-MiniLM-L3-v2",
    epochs=EPOCHS, batch_size=TRAIN_BS, lr=LR, max_len=96
)

# (4) Bi-encoder: Multiple Negatives Ranking Loss
mn_model, mn_val_f1, mn_thr = run_biencoder(
    loss_type="mnr",
    base_model="thenlper/gte-small",
    epochs=EPOCHS, batch_size=TRAIN_BS, lr=LR, max_len=96
)

[0-baseline:sentence-transformers/all-MiniLM-L6-v2] Val F1=0.7372 @ thr=0.733


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.174000
1000,0.137200
1500,0.127800
2000,0.122400
2500,0.118800
3000,0.109600
3500,0.109500
4000,0.107000
4500,0.106700
5000,0.105600


[bi-cos:sentence-transformers/multi-qa-MiniLM-L6-cos-v1] Val F1=0.8219 @ thr=0.560


...

[bi-contrastive:sentence-transformers/paraphrase-MiniLM-L3-v2] Val F1=0.8268 @ thr=0.804


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.248400
1000,0.078100
1500,0.070800


[bi-mnr:thenlper/gte-small] Val F1=0.7673 @ thr=0.801


In [13]:
ce_name, ce_val_f1, ce_thr = run_cross_encoder(
    model_name="cross-encoder/ms-marco-MiniLM-L-6-v2",
    epochs=EPOCHS, batch_size=64, warmup_ratio=0.1, max_len=96
)

[cross] training on 323,429 pairs | steps/epoch=5054 | warmup_steps=1010


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Token indices sequence length is longer than the specified maximum sequence length for this model (182 > 96). Running this sequence through the model will result in indexing errors


Step,Training Loss
500,0.556700
1000,0.384700
1500,0.345200
2000,0.323300
2500,0.311400
3000,0.288000
3500,0.283900
4000,0.276300
4500,0.276800
5000,0.270100


Batches:   0%|          | 0/316 [00:00<?, ?it/s]

[cross-encoder:cross-encoder/ms-marco-MiniLM-L-6-v2] Val F1=0.8492 @ thr=-0.560


In [14]:
# Summary
summary = pd.DataFrame([
    ["1-baseline (no FT)", base_name,                         base_val_f1, base_thr],
    ["2-bi CosineLoss",    "sentence-transformers/multi-qa-MiniLM-L6-cos-v1", cos_val_f1,  cos_thr],
    ["3-bi Contrastive",   "sentence-transformers/paraphrase-MiniLM-L3-v2",   ctr_val_f1,  ctr_thr],
    ["4-bi MNRL",          "thenlper/gte-small",                              mn_val_f1,   mn_thr],
    ["5-cross-encoder",    ce_name,                                           ce_val_f1,   ce_thr],
], columns=["experiment","backbone","val_F1","val_threshold"]).sort_values("val_F1", ascending=False)

print("\n Summary")
print(summary.to_string(index=False))


 Summary
        experiment                                        backbone   val_F1  val_threshold
   5-cross-encoder            cross-encoder/ms-marco-MiniLM-L-6-v2 0.849224      -0.559669
  3-bi Contrastive   sentence-transformers/paraphrase-MiniLM-L3-v2 0.826775       0.804319
   2-bi CosineLoss sentence-transformers/multi-qa-MiniLM-L6-cos-v1 0.821893       0.560347
         4-bi MNRL                              thenlper/gte-small 0.767340       0.801405
1-baseline (no FT)          sentence-transformers/all-MiniLM-L6-v2 0.737199       0.733384
